# Question 1: 
On Shopify, we have exactly 100 sneaker shops, and each of these shops sells only one model of shoe. We want to do some analysis of the average order value (AOV). When we look at orders data over a 30 day window, we naively calculate an AOV of $3145.13. Given that we know these shops are selling sneakers, a relatively affordable item, something seems wrong with our analysis. 

1.a) Think about what could be going wrong with our calculation. Think about a better way to evaluate this data. 
<br>
1.b) What metric would you report for this dataset?
<br>
1.c) What is its value?


In [ ]:
import pandas as pd

In [2]:
data = pd.read_excel('2019 Winter Data Science Intern Challenge Data Set.xlsx')

In [4]:
data.describe()

,order_id,shop_id,user_id,order_amount,total_items
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000
mean,2500.500000,50.078800,849.092400,3145.128000,8.78720
std,1443.520003,29.006118,87.798982,41282.539349,116.32032
min,1.000000,1.000000,607.000000,90.000000,1.00000
25%,1250.750000,24.000000,775.000000,163.000000,1.00000
50%,2500.500000,50.000000,849.000000,284.000000,2.00000
75%,3750.250000,75.000000,925.000000,390.000000,3.00000
max,5000.000000,100.000000,999.000000,704000.000000,2000.00000


# Answer: 1.a) 
Upon inspecting the data we see that the AOV Value has been obtained by averaging the order_amount data which clearly has an outlier value as the maximum value of the order_amount is $704000 which is very high for a pair of sneakers. Since the calculation has really high values to obtain a realistic model we should consider using a metrics which is less affected by the presence of outlier. Since mean is highly affected with the presence of outliers in a dataset it is not be a good metric for us to evaluate the average order value as it is not conveying the true picture. 

The shopify blog also mentions that to calculate AOV we should first calculate the total revenue and divide it by total number of orders for each business. Therefore, before using median and mode of the order amount data we will first calculate the revised AOV using this information

# Answer: 1.b)
To get a better picture of what is going on we should focus on metrics like median and mode as these values are less affected by the outliers. Apart from this, the shopify blog - https://www.shopify.com/blog/average-order-value - also emphasizes on the importance of using median and mode values for different orders as they are a better metric in understanding and subsequently growing a business.

In [10]:
total_revenue_shopid = data.groupby(['shop_id'])['order_amount'].agg('sum')
total_revenue_shopid.name ='total_revenue'
total_orders_shopid = data.groupby(['shop_id'])['total_items'].agg('sum')
total_orders_shopid.name = 'total_orders'
data_shopid = pd.concat([total_revenue_shopid, total_orders_shopid], axis=1)
aov_shopid = data_shopid['total_revenue']/data_shopid['total_orders'] 
aov_shopid.name = 'AOV'
data_shopid = pd.concat([total_revenue_shopid, total_orders_shopid , aov_shopid], axis=1)
data_shopid

,total_revenue,total_orders,aov
shop_id,,,
1,13588,86,158.0
2,9588,102,94.0
3,14652,99,148.0
4,13184,103,128.0
5,13064,92,142.0
...,...,...,...
96,16830,110,153.0
97,15552,96,162.0
98,14231,107,133.0


In [12]:
data_shopid.describe()

,total_revenue,total_orders,aov
count,1.000000e+02,100.000000,100.000000
mean,1.572564e+05,439.360000,407.990000
std,1.216218e+06,3396.366111,2557.462906
min,6.840000e+03,67.000000,90.000000
25%,1.293050e+04,88.000000,132.750000
50%,1.488750e+04,100.000000,153.000000
75%,1.760000e+04,111.250000,168.250000
max,1.199018e+07,34063.000000,25725.000000


We observe that the revised AOV calculated as per Shopify's blog has come down to $407.9 which is still a realistic number as opposed to the $3145.128 value previously observed. 

However, we should still focus on median values for the purpose of our analysis since it is less likely to be affected by outliers.

Before calculating the median value we should first inspect further on the outlier values, and then calculate the median and mode values for it.

In [13]:
data.sort_values(by=['order_amount'], ascending= False)

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
2153,2154,42,607,704000,2000,credit_card,2017-03-12 04:00:00.000
3332,3333,42,607,704000,2000,credit_card,2017-03-24 04:00:00.000
520,521,42,607,704000,2000,credit_card,2017-03-02 04:00:00.000
1602,1603,42,607,704000,2000,credit_card,2017-03-17 04:00:00.000
60,61,42,607,704000,2000,credit_card,2017-03-04 04:00:00.000
...,...,...,...,...,...,...,...
3871,3872,92,818,90,1,debit,2017-03-18 09:10:07.550
2503,2504,92,935,90,1,credit_card,2017-03-28 02:30:25.553
4414,4415,92,927,90,1,credit_card,2017-03-17 09:57:01.115
3363,3364,92,730,90,1,credit_card,2017-03-11 23:20:31.493


From the above result, we observe that shop number 42 has the highest order amounts with multiple transaction of $704000 made by credit card by the same user (id: 607). These orders should be checked further since they have been made in the same month by the same user.

##### MEDIAN and MODE Calculation 

In [17]:
print("Median AOV value: $",data_shopid['aov'].median())
print("Mode AOV value: $",data_shopid['aov'].mode()[0])

Median AOV value: $ 153.0
Mode AOV value: $ 153.0


# Answer 1.c) 
Therefore for the purpose of our analysis since both the metrics have same value of $153, we can use either of them to obtain an idea of the average order values. My recommendation however would be to use the median values since it gives us a better understanding of data and is not affected by outliers.

# Question 2: 

For this question you’ll need to use SQL. Follow this link to access the data set required for the challenge. Please use queries to answer the following questions. Paste your queries along with your final numerical answers below.

## Q2.a) How many orders were shipped by Speedy Express in total?
## Answer:=> Total orders shipped by Speedy Express = 54

### Query: 
SELECT COUNT(o.ShipperID)
FROM Orders AS o
WHERE o.ShipperID IN (SELECT ShipperID 
    FROM Shippers AS s
    WHERE s.ShipperName LIKE "Speedy Express")

## Q2.b)What is the last name of the employee with the most orders?
## Answer=> Last Name of the Employee with most orders is Peacock

### Query:
SELECT TOP 1 e.LastName 
FROM Orders o INNER JOIN Employees e 
ON o.EmployeeID=e.EmployeeID
GROUP BY e.LastName 
ORDER BY Count(o.OrderID) DESC;


## Q2.c) What product was ordered the most by customers in Germany?
## Answer=> Boston Crab Meat was the product that was most ordered by customers in Germany with 160 orders in total

### Query:
SELECT TOP 1 p.ProductName,SUM(od.Quantity) as total_quant
FROM (((Products as p
LEFT JOIN OrderDetails as od
ON p.ProductID = od.ProductID)
LEFT JOIN Orders as o
ON od.OrderID = o.OrderID)
LEFT JOIN Customers as c
ON o.CustomerID = c.CustomerID)
WHERE c.Country = "Germany"
GROUP BY p.ProductName
ORDER BY SUM(od.Quantity) DESC
